In [5]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

import models.TextRNN_Att as model
from utils import DataConfig

np.random.seed(3407)
torch.manual_seed(3407)
torch.cuda.manual_seed_all(3407)
torch.backends.cudnn.deterministic = True  # 保证每次结果一样
torch.backends.cudnn.benchmark = False
UNK, PAD = '<UNK>', '<PAD>'  # 未知字，padding符号

In [6]:
data_config = DataConfig('100d', 'word2vec')
model_config = model.ModelConfig()

In [95]:
class Model(nn.Module):
    def __init__(self, model_config, data_config):
        super(Model, self).__init__()
        if data_config.embedding_pretrained is not None:
            self.embedding = nn.Embedding.from_pretrained(data_config.embedding_pretrained, freeze=False)
        else:
            self.embedding = nn.Embedding(data_config.n_vocab, data_config.embed, padding_idx=data_config.n_vocab - 1)
        self.lstm = nn.LSTM(data_config.embed, model_config.hidden_size, model_config.num_layers,
                            bidirectional=True, batch_first=True, dropout=model_config.dropout)
        # self.u = nn.Parameter(torch.Tensor(config.hidden_size * 2, config.hidden_size * 2))
        self.w = nn.Parameter(torch.rand(model_config.hidden_size * 2))
        self.fc = nn.Linear(model_config.hidden_size * 2, data_config.num_classes)

    def forward(self, x):
        emb_x = self.embedding(x)  # [batch_size, seq_len, embeding]=[128, 32, 300]
        H, _ = self.lstm(emb_x)  # [batch_size, seq_len, hidden_size * num_direction]=[128, 32, 256]
        M = F.tanh(H)  # [128, 32, 256]
        alpha = F.softmax(torch.matmul(M, self.w), dim=1).unsqueeze(-1)  # [128, 32, 1]
        out = H * alpha  # [128, 32, 256]
        out = torch.sum(out, dim=1)  # 求和
        out = F.relu(out)
        out = self.fc(out)  # [128, 64]
        return out


In [96]:
model = Model(model_config, data_config).to(data_config.device)

In [97]:
model(torch.randint(1, 10, [2, 30]).to(data_config.device)).size()


torch.Size([2, 5])

In [42]:
def init_network(model, method='xavier', exclude='embedding'):
    for name, w in model.named_parameters():
        if exclude not in name:  # 如果不是嵌入层
            if 'weight' in name:  # weight 三种初始化方式
                if method == 'xavier' and len(w.size() < 2):
                    nn.init.xavier_normal_(w)
                elif method == 'kaiming':
                    nn.init.kaiming_normal_(w)
                else:
                    nn.init.normal_(w)
            elif 'bias' in name:  # bias 置0
                nn.init.constant_(w, 0)
            else:
                pass

In [23]:
init_network(model)

In [76]:
from torchinfo import summary

summary(model, input_size=(1, 30), dtypes=[torch.long])

Layer (type:depth-idx)                   Output Shape              Param #
Model                                    [1, 5]                    512
├─Embedding: 1-1                         [1, 30, 100]              72,700
├─LSTM: 1-2                              [1, 30, 512]              2,310,144
├─Tanh: 1-3                              [1, 30, 512]              --
├─Linear: 1-4                            [1, 5]                    2,565
Total params: 2,385,921
Trainable params: 2,385,921
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 69.38
Input size (MB): 0.00
Forward/backward pass size (MB): 0.15
Params size (MB): 9.54
Estimated Total Size (MB): 9.69

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# 读取CSV文件
df = pd.read_csv('./ship_data/experiment_data.csv')  # 替换为你的CSV文件路径

# 划分数据集
train_df, temp_df = train_test_split(df, test_size=0.1, random_state=3407)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=3407)

# 保存划分后的数据集为新的CSV文件
train_df.to_csv('train_dataset.csv', index=False)
val_df.to_csv('val_dataset.csv', index=False)
test_df.to_csv('test_dataset.csv', index=False)
